In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate as integrate
from scipy.integrate import quad
from scipy.special import kn
from scipy.special import kv

In [2]:
# Values of parameters taken from literature (cited in the report)

kpc = 3.086e21 # One kiloparsec in cm
k_B = 1.38e-16
c = 3e10 # Speed of light light in cm/s
m_e = 1e-27 # Mass of electron in g
e = 5e-10 # Charge of electron in esu
M_sun = 1.989e33  # Mass of sun in g
h = 6.626e-27 #Planck's constant in cgs
G = 6.67e-8 #Gravitational constant in cgs
sigma_thomson = 6.652e-25 #Thomson scattering cross section in cm^2
B = 33 #Magnetic Field of the corona, assumed to be constant
D = 2.39 * kpc #Distance of V404 Cyg from Earth
r_g = G * 9 * M_sun / c**2 #gravitational radius of V404 Cyg, its mass has been obtained from literature
yr = 365 * 24 * 60 ** 2 # One year in s

#Values of parameters that have been adjusted to obtain best fits

gamma_min = 1.1 #Minimum lorentz factor of electrons in the corona
gamma_max = 100 #Maximum lorentz factor of electrons in the corona
T_corona = 1e8 # Temperature of the corona in Kelvin
T_disk = 1e7 # Temperature of the disk in Kelvin
n = 1e15 #Electron number density of corona (we are using the value obtained from equipartition calculations)
R = 10 * r_g #Radius of the corona
v = 0.5 * c #Velocity of bulk flow through the corona

<h2>Sampling From a Planck Function (Shashanth)</h2>





In [3]:
def planck_new(T, nu):
  """
  Defines a new planck function independent of the MC tutorial code.

  Parameters
  ----------------
  T: float
  - Temperature of the black body in Kelvin
  nu: float or numpy array
  - Frequency/ies for which the planck function needs to be calculated

  Returns
  ----------------
  B_nu: float or array
  - The planck function values corresponding the given value of T and nu(s)
  """
  B_nu = 8 * np.pi * h * nu**3 / c**2 * 1 / (np.exp(h*nu/(k_B * T)))
  return B_nu

def conv_to_cdf(f):
    """
    Converts a given function f(x) into a pdf and then into a cdf.

    Parameters
    ----------------
    f: numpy array
    - The function to be converted in the form of an array

    Returns
    ----------------
    f_cdf: numpy array
    - The cummulative distribution function corresponding to f
    """
    f_pdf = f / np.sum(f) # Can convert f(x) into a pdf by normalizing
    f_cdf = np.cumsum(f_pdf)
    return f_cdf

def sample_func(f_cdf, x, n_samples=1):
    """
    Samples a given cummulative distribution function with the corresponding 'x' values.

    Parameters
    ----------------
    f_cdf: numpy array
    - Cummulative distribution function
    x: numpy array
    - The x values that correspond to the cummulative distribution function
    n_samples: int
    - The number of samples to be taken

    Returns
    ----------------
    y_val: float or numpy array
    - The sampled value
    """
    sample = np.random.uniform(size=n_samples)
    y_val = np.interp(sample, f_cdf, x)
    return y_val

def wrapper_sample_planck(mc_parms, number=1):
    """
    Wrapper function that samples one frequency from the planck function and returns the corresponding energy value

    Parameters
    ----------------
    mc_parms: dict
    - Dictionary of required values
    number: int
    - Number of samples to be taken

    Returns
    ----------------
    sampled_hnu: float or numpy array
    """
    nus = np.logspace(3,20, 1000)
    f = planck_new(mc_parms['T_BB'], nus)
    f_cdf = conv_to_cdf(f)
    sampled_nu = sample_func(f_cdf, nus, number)
    sampled_hnu = sampled_nu * h
    return sampled_hnu

def calc_planck(T):
    """
    Calculates the planck function for pre-defined frequency range
    Parameters
    --------------
    T: float
    Temperature of the BB in Kelvin

    Returns
    ----------------
    nus: numpy array
    - Frequency range in Hz in which the planck function has been computed
    f: numpy array
    - Value of the planck function at the frequencies output
    """
    nus = np.logspace(3,20, 1000)
    f = planck_new(mc_parms['T_BB'], nus)
    return nus, f
# plot_planck(5000)

nu = np.logspace(9, 19, 1000) # Frequency range to integrate over

N_photons_actual = np.trapz(planck_new(T_disk, nu), nu) * 4 * np.pi**2 * (R)**2 # Integrate planck function

<h2>Sampling From a Maxwell-Juttner Distribution (Shashanth)</h2>

In [4]:
def maxwell_juttner(gamma_inp):
  """
  The Maxwell-Jutner distribution defined in terms of gamma


  """
  theta = (k_B * T_corona) / (m_e * c**2)
  # theta = 1
  beta = np.sqrt(1 - (1/gamma_inp**2))
  f = (gamma_inp**2 * beta) / (theta * kn(4, 1/theta)) * np.exp(-(gamma_inp/theta))
  # print('Theta:', theta)
  # print('Beta:', beta)
  return f

def E_dist_maxwell_juttner():
  """
  Function to test maxwell juttner distribution. Uses predefined functions and values
  Parameters
  -----------------
  -
  Returns
  -----------------
  None
  """
  gamma_range = np.logspace(np.log10(1.000001), np.log10(gamma_max), 10000)
  max_jutt = maxwell_juttner(gamma_range)
  # plt.plot(gamma_range, max_jutt)
  # plt.title('Initial Maxwell-Juttner Distribution')
  # plt.xscale('log')
  # plt.yscale('log')
  # plt.xlabel('gamma')
  # plt.ylabel('f (gamma)')
  # plt.show()

  sampled_vel, sampled_gamma = sample_maxwell_juttner(10000, mc_parms)
  # plt.hist(sampled_vel, bins=30)
  # plt.title('Sampled Velocity Distribution')
  # plt.xlabel('v (cm/s)')
  # plt.ylabel('Frequency (number)')
  # plt.show()
  plt.hist(sampled_gamma * m_e * c**2, bins=30)
  # x = np.linspace(1e-6, 1.5*1e-6, 100)
  # y = x**(-5.5)
  # y = y/(y[1]/3000)
  # plt.plot(x, y)
  plt.title('Sampled Energy Distribution')
  plt.xlabel('Energy (ergs)')
  plt.ylabel('Frequency (number)')
  plt.show()

# The maxwell-juttner distribution is already in the form of a probability distribution but is defined for gamma ranges from 0 to infinity.
# We need to normalize it for our range of gamma and then convert it into a cdf.

def maxwell_juttner_cdf(maxwell_juttner, gamma_range):
  """
  Converts the maxwell juttner function into a cdf that we can sample for our gamma range. Returns value(s) of gamma
  Returns
  -------------
  """

  pdf = maxwell_juttner(gamma_range)
  pdf = pdf / sum(pdf)
  cdf = np.cumsum(pdf)
  return cdf

def sample_maxwell_juttner(n_samples, mc_parms):
  """
  Returns sampled velocities from the maxwell-juttner distribution

  Parameters
  -------------
  n_samples: int
  - Number of samples to be conducted
  mc_parms: dict
  - Dictionary with pre-defined values

  Returns
  -------------
  velocity_sampled: float or numpy array
  - sampled value(s) of velocity
  gamma_sampled: float or numpy array
  - sampled value(s) of gamma
  """
  nbins_gamma = 100000
  gamma_range = np.logspace(np.log10(mc_parms['gamma_min']), np.log10(mc_parms['gamma_max']), nbins_gamma)
  gamma_range = np.linspace(mc_parms['gamma_min'], mc_parms['gamma_max'], nbins_gamma)
  # print(gamma_range)
  cdf = maxwell_juttner_cdf(maxwell_juttner, gamma_range)

  samples = np.random.uniform(size=n_samples)
  gamma_sampled = np.interp(samples, cdf, gamma_range)

  # Test Plots
  # plt.plot(gamma_range, cdf)
  # plt.hist(gamma_sampled, bins=300)
  # plt.xlim(1, 1e4)

  # Convert gamma into velocity
  velocity_sampled = c * np.sqrt(1 - 1/gamma_sampled**2)

  return velocity_sampled, gamma_sampled # Return sampled velocity

def wrapper_sample_maxwell_juttner(mc_parms):
  """
  Wrapper that allows us to use the sampling function with the Monte Carlo code.

  Parameters
  -------------
  mc_parms: dict
  - Dictionary with pre-defined values

  Returns
  -------------
  vel: float
  - Sampled velocity
  """
  vel,gamma = sample_maxwell_juttner(1, mc_parms)
  return vel

<h2>Code From MC Tutorial (From Sebastian Heinz)</h2>



In [5]:
c_light=c
m_electron=m_e
sigma_t=6.65e-25
figure_counter=0

def compton_y(pre,post):
    """
    Calculate the Compton Y value based on pre and post scattering values.
    This function computes the mean change ratio between pre and post values.

    Parameters:
    ------------
    pre : array_like
        Array containing pre-scattering values.
    post : array_like
        Array containing post-scattering values. Should be the same length as pre.

    Returns:
    ---------
           float
           Compton Y value representing the mean change ratio between pre and post values.
    """
    y = np.mean((post-pre)/pre)
    return(y)

def random_direction(number=None):
    """Returns randomly oriented unit vectors.

    Args:
        None

    Parameters:
        number: number of random vectors to return

    Returns::
        (number,3)-element numpy array: Randomly oriented unit vectors
    """

    #
    # This is how you draw a random number for a uniform
    # distribution:
    #

    if number is None:
        number=1

    phi=2.*np.pi*np.random.rand(number)
    cos_phi=np.cos(phi)
    sin_phi=np.sin(phi)
    cos_theta=2.*np.random.rand(number)-1
    sin_theta=np.sqrt(1 - cos_theta**2)
    return((np.array([sin_theta*cos_phi,sin_theta*sin_phi,cos_theta])).transpose())

def photon_origin(number=None):
    """Returns emission location of a photon
    """
    if number is None:
        number=1
    return(np.zeros([number,3]))

def draw_seed_photons(mc_parms,number=None):
    """Returns a single seed photon

    Args:
        mc_parms (dictionary): MC parameters

    Parameters:
        number (integer): number of photons to return

    Returns:
        (number x 4)numpy array: Photon momentum 4-vectors
        (number x 3)numpy array: Initial photon positions
    """
    if number is None:
        number=1
    x_seed=photon_origin(number=number)
    n_seed=random_direction(number=number)
    hnu=mc_parms['hnu_dist'](mc_parms,number=number)
    p_seed=(np.array([hnu,hnu*n_seed[:,0],hnu*n_seed[:,1],hnu*np.abs(n_seed[:,2])])).transpose()/c_light
    return(p_seed,x_seed)

def tau_of_scatter():
    """Calculates optical depth a photon traveled to before interacting, given probability

    Args:
        None

    Returns:
        real: Optical depth as function of P

    """
    return(-np.log(np.random.rand()))

def distance_of_scatter(mc_parms):
    """Calculates the distance that corresponds to an optical depth tau

    Args:
        tau (real): optical depth photon travels before scattering occurs
        mc_parsm (dictionary): MC parameters

    Returns:
        real: distance

    """
    tau=tau_of_scatter()
    electron_density=mc_parms['tau']/mc_parms['H']/sigma_t
    distance=tau/sigma_t/electron_density
    return(distance)

def scatter_location(x_old,p_photon,mc_parms):
    """This function goes through the steps of a single scattering

    Args:
        x_old.   (three-element numpy array): holds the position
        p_photon (four-element numpy array): the in-coming photon four-momentum
        mc_parms (dictionary): the simulation parameters

    Returns:
        three-element numpy array: scattering location
    """

    # ...path-length:
    distance = distance_of_scatter(mc_parms)
    # ...in direction:
    photon_direction=p_photon[1:]/p_photon[0]
    # Update photon position with the new location
    x_new = x_old + distance*photon_direction
    return(x_new)


def draw_electron_velocity(mc_parms,p_photon):
    """Returns a randomized electron velocity vector for inverse
       Compton scattering, taking relativistic foreshortening of the
       photon flux in the electron frame into account

       Args:
           mc_parms (dictionary): Monte-Carlo parameters
           p_photon (4 dimentional np array): Photon 4-momentum

       Returns:
           3-element numpy array: electron velocity
    """
    v=mc_parms['v_dist'](mc_parms)
    n=draw_electron_direction(v,p_photon)
    return(v*n)

def draw_electron_direction(v,p_photon):
    """Draw a randomized electron direction, taking account of the
       increase in photons flux from the foward direction, which
       effectively increases the cross section for forward scattering.

       Args:
            v (real): electron speed
            p_photon (4 element numpy array): photon 4-momentum

       Returns:
           3-element numpy array: randomized electron velocity vector
    """
    phi=2.*np.pi*np.random.rand()
    cosp=np.cos(phi)
    sinp=np.sin(phi)
    cost=mu_of_p_electron(v/c_light,np.random.rand())
    sint=np.sqrt(1 - cost**2)

    n_1=p_photon[1:]/p_photon[0]
    if (np.sum(np.abs(n_1[1:2])) != 0):
        n_2=np.cross(n_1,np.array([1,0,0]))
    else:
        n_2=np.cross(n_1,np.array([0,1,0]))
    n_2/=np.sqrt(np.sum(n_2**2))
    n_3=np.cross(n_1,n_2)

    # express new vector in old base
    n_new=(n_2*cosp+n_3*sinp)*sint + n_1*cost
    return(n_new/np.sqrt(np.sum(n_new**2)))

def mu_of_p_electron(beta,p):
    """Invert probability for foreshortened effective
       Thomson scattering cross section, with

       P =

       Args:
           beta (real): v/c for electron
           p: probability value between 0 and 1

       Returns:
           real: cos(theta) relative to photon direction
    """
    mu=1/beta-np.sqrt(1/beta**2 + 1 - 4*p/beta + 2/beta)
    return(mu)

def lorentz_transform(p,v):
    """Returns general Lorentz transform

    Args:
        p (four-element numpy array): input four-vector
        v (three-element numpy array): the 3-velocity of the frame we want to transform into

    Returns:
        four-element numpy array: the transformed four-vector
    """
    beta=np.sqrt(np.sum(v**2))/c_light
    beta_vec=v/c_light
    gamma=1./np.sqrt(1. - beta**2)
    matrix=np.zeros((4,4))
    matrix[0,0]=gamma
    matrix[1:,0]=-gamma*beta_vec
    matrix[0,1:]=-gamma*beta_vec
    matrix[1:,1:]=(gamma-1)*np.outer(beta_vec,beta_vec)/beta**2
    for i in range(1,4):
        matrix[i,i]+=1
    return(np.dot(matrix,p))

def cos_theta_thomson(p):
    """Invert P(<\theta) to calculate cos(theta)

        Args:
            p (real): probability between 0 and 1

        Returns:
            real: scattering angle drawn from Thomson distribution
    """

    a=-4 + 8*p
    b=a**2 + 4
    return((np.power(2,1/3)*np.power(np.sqrt(b)-a,2/3)-2)/
           (np.power(2,2/3)*np.power(np.sqrt(b)-a,1/3)))

def thomson_scatter(p_photon):
    """This function performs Thomson scattering on a photon

    Args:
        p_photon (4-element numpy array): Incoming photon four-vector

    Returns:
        4-element numpy array: Scattered photon four-vector
    """
    n_1=p_photon[1:]/p_photon[0]
    if (np.sum(np.abs(n_1[1:2])) != 0):
        n_2=np.cross(n_1,np.array([1,0,0]))
    else:
        n_2=np.cross(n_1,np.array([0,1,0]))
    n_2/=np.sqrt(np.sum(n_2**2))
    n_3=np.cross(n_1,n_2)

    # scattering is uniform in phi
    phi=2.*np.pi*np.random.rand()
    cosp=np.cos(phi)
    sinp=np.sin(phi)

    # draw cos_theta from proper distribution
    cost=cos_theta_thomson(np.random.rand())
    sint=np.sqrt(1 - cost**2)

    # express new vector in old base
    n_new=(n_2*cosp+n_3*sinp)*sint + n_1*cost
    n_new/=np.sqrt(np.sum(n_new**2))

    # return scatterd 4-momentum vector
    return(np.array(p_photon[0]*np.array([1,n_new[0],n_new[1],n_new[2]])))

def inverse_compton_scatter(p_photon,mc_parms):
    """This function performs an iteration of inverse Compton scattering off an electron of velocity v_vec.

    Args:
        p_photon (four element numpy array): input photon four-momentum
        v_vec (three element numpy array): 3-velocity vector of the scattering electron

    Returns:
        four-element numpy array: scattered photon four-momentum in observer's frame
    """

    # throw the dice one more time to draw a random electron velocity

    velocity=draw_electron_velocity(mc_parms,p_photon)
    # first, transform to electron frame
    p_photon_prime=lorentz_transform(p_photon,velocity)
    # Thomson scatter
    p_out_prime=thomson_scatter(p_photon_prime)
    # transform back to observer frame
    return(lorentz_transform(p_out_prime,-velocity))

def monte_carlo(mc_parms):
    """Perform a simple Monte-Carlo simulation

    Args:
       mc_parms (dictionary): Monte-Calro parameters

    Returns:
        numpy array: List of escaped photon energies
        numpy array: Lost of seed energies of all escaping photons
    """

    # arrays to store initial and final photon energies
    hnu_seed=np.zeros(mc_parms['n_photons'])
    hnu_scattered=hnu_seed.copy()

    # draw our seed-photon population. Much faster to do this once for all photons
    p_photons,x_photons=draw_seed_photons(mc_parms,number=mc_parms['n_photons'])

    # run the scattering code n_photons times
    for p_photon,x_photon,i in zip(p_photons,x_photons,range(mc_parms['n_photons'])):
        # initial photon four-momentum
        # store seed photon energy for future use (calculating Compton-y parameter)
        hnu_seed[i]=p_photon[0]*c_light
        # keep scattering until absorbed or escaped
        scattered=True

        while (scattered):
            # find next scattering location
            x_photon = scatter_location(x_photon,p_photon,mc_parms)

            # if it's inside the corona, perform inverse Compton scatter
            if (x_photon[2]>=0 and x_photon[2]<=mc_parms['H']):
                p_photon=inverse_compton_scatter(p_photon,mc_parms)
            else:
                scattered=False
                if (x_photon[2]<=0):
                    p_photon*=0
        # store the outgoing photon energy in the array
        hnu_scattered[i]=p_photon[0]*c_light
    # only return escaped photons and their seed energy
    return(hnu_scattered[hnu_scattered > 0],hnu_seed[hnu_scattered > 0])

def plot_mc(mc_parms,bins=None,xlims=None,filename=None):
    """Run an MC simulation and plot a histogram of the output. Saves a pdf of the plot

    Args:
        mc_parms (dictionary): Monte-Carlo parameters

    Paramters:
        bins (numpy array): Optional spectral bins
        xlims (2-element list, real): plot-limits

    Returns:
        numpy array: The energies of all photons escaping the corona
        numpy array: The seed-energies of the escaping photons
    """
    global figure_counter

    if (xlims is None):
        xlims=[0.1,100]
    if (bins is None):
        bins=np.logspace(np.log10(xlims[0]),np.log10(xlims[1]),num=100)
    if (filename is None):
        filename='inverse_compton_MC_{0:d}.pdf'.format(figure_counter)

    # Now run simulation and normalize all outgoing photon energies
    # so we can investigate energy gains and losses

    hnu_scattered,hnu_seeds=np.array(monte_carlo(mc_parms))  # / mc_parms['kt_seeds']

    # Commented out below to remove plots

    # fig=plt.figure()
    # plt.hist(hnu_scattered,bins=bins,log=True,
    #          label=r'$\tau=${:4.1f}'.format(mc_parms['tau']))
    # plt.xscale('log')
    # # plt.xlim(xlims[0],xlims[1])
    # plt.xlabel(r'$h\nu/h\nu_{0}$')
    # plt.ylabel(r'$N(h\nu)$')
    # plt.legend()
    # fig.savefig(filename)
    # figure_counter+=1
    # plt.show()
    print('Fraction of escaping photons: {0:5.3e}\n'.format(hnu_scattered.size/mc_parms['n_photons']))
    print('Compton y parameter: {0:5.3e}\n'.format(compton_y(hnu_seeds,hnu_scattered)))
    return(hnu_scattered,hnu_seeds)

<h2>Cyclotron and Synchrotron Emission (Raghavan)</h2>

In [6]:
def calc_Ledd():
    '''
    This function calculates the accretion rate in erg/s as a fraction of Eddington luminosity
    The Eddington luminosity is given by L_edd = 1.26 x 10^38 M/M_sun erg/s where M is the mass of the black hole in solar masses
    '''
    L_edd = 1.26 * 1e38 * 9
    f = 1e-4 # In quiescence, accretion rate is 1e-4 of L_edd
    return f * L_edd

def calc_etaMdot():
    '''
    This function calculates the mass accretion rate of the black hole in g/s.
    Returns:
    --------
    eta: float
         Efficiency of accretion
    Mdot: float
          Mass accretion rate in g/s
    '''
    Accretion = calc_Ledd()
    Mdot = Accretion / c**2
    eta = 1
    return eta, Mdot

def calc_Lj(eta, Mdot):
    '''
    This calculates the luminosity at the base of the jet (here, the corona).
    We are assumming it is essentially equal to the fraction of the Eddington luminosity that corresponds to the accretion rate.

    Parameters:
    ------------
    eta: float
         Efficiency of accretion
    Mdot: float
          Mass accretion rate in g/s

    Returns:
    ---------
    L_corona: float
              Luminosity at the base of the jet/corona (in erg/s)
    '''
    eta, Mdot = calc_etaMdot()
    L_corona = eta * Mdot * c**2
    return L_corona

def calc_Utot(eta, Mdot):
    '''
    Computes the total energy density in the corona using the formula:
    U_tot = Luminosity at corona/(Area*velocity of accreting matter)

    Parameters:
    ------------
    eta: float
         Efficiency of accretion
    Mdot: float
          Mass accretion rate in g/s

    Returns:
    ---------
    U_tot: float
           Energy density of the corona in erg/cm^3
    '''
    L_j = calc_Lj (eta, Mdot)
    R = 10 * r_g
    U_tot = L_j / (4 * np.pi * R**2 * v)
    return U_tot

def calc_B(eta, Mdot):
    '''
    This function calculates the magnetic field in the corona assuming equipartition division of energy
    holds between the electron energy and magnetic field energy densities.

    Parameters:
    ------------
    eta: float
         Efficiency of accretion
    Mdot: float
          Mass accretion rate in g/s

    Returns:
    ---------
    B: float
       Magnetic field strength in the corona in gauss
    '''
    Utot = calc_Utot(eta, Mdot)
    Bsq = 4 * np.pi * Utot
    return np.sqrt(Bsq)

def get_Q(gamma):
    '''
    Obtains the dimensionless momentum required for describing particles in the Maxwell-Juttner distribution in momentum space.
    γ = sqrt (Q^2+1) where Q = p/(m_e * c)

    Parameters:
    ------------
    gamma: float
           electron lorentz factor

    Returns:
    -----------
    The dimensionless momentum in terms of the lorentz factor
    '''
    return np.sqrt(gamma**2 - 1)

def get_gammaQ(gamma):
    '''
    Returns gamma in terms of the dimensionless momentum.
    May seem a bit redundant, but it is useful for computing eq6 from the Bhjet paper (Lucchini et al.).

    Parameters:
    Parameters:
    ------------
    gamma: float
           electron lorentz factor

    Returns:
    -----------
    gammaQ :The lorentz factor in terms of the dimensionless momentum

    '''
    Q = get_Q(gamma)
    gammaQ = np.sqrt(Q**2 + 1)
    return gammaQ

def get_NQ(gamma):
    '''
    This function computes the distribution of thermal particles in momentum space assuming a Maxwell-Juttner distribution holds.
    It essentially performs the eq6 from the Bhjet paper (Lucchini et al.) with the simplification in normalization also given by the authors.

    Parameters:
    ------------
    gamma: float, ideally numpy.ndarray
           electron lorentz factor

    Returns:
    -----------
    NQ: Distribution of thermal particles in momentum space (will be a constant if only 1 gamma given,
    an array equivalent to the distribution if gamma is an array)
    '''
    Q = get_Q(gamma)
    gammaQ = get_gammaQ(gamma)
    theta = k_B * T_corona/ (m_e *c**2) #temperature of leptons in units of m_e*c^2
    N0 = n/ (m_e * c**3 * theta * kv(2,1/theta))  #kv is modified Bessel function of second kind
    expterm = np.exp(-gammaQ/theta)
    NQ = N0 * Q**2 * expterm
    return NQ

def get_dQdgamma(gamma):
    '''
    This function calculates the derivative of dimensionless momentum with respect to the lorentz factor of the electrons.
    It is needed to implement equation 2 in Lucchini et al. (to go from momentum to gamma space)

    Parameters:
    ------------
    gamma: float
           electron lorentz factor

    Returns:
    -----------
    Derivative of Q with respect to gamma, float
    '''
    return gamma/np.sqrt(gamma**2 - 1)

def calc_Larmorfreq():
    '''
    Returns the Larmor frequency (float) for cyclotron radiation.
    No input parameters as B is a constant given by literature.
    '''
    return e*B/(2*np.pi*m_e*c)

def calc_magneticU():
    '''
    Returns magnetic field energy density in erg/cm^3
    Does not take an input as B is the constant obtained from literature.
    '''
    return B**2/(8 * np.pi)

def const_cycloemissivity():
    '''
    This is an intermediary function that computes constants in equation 16 of Lucchini et al. used for finding the cyclotron emissivity.
    '''
    U_B = calc_magneticU()
    nu1 = calc_Larmorfreq()
    const = 8/3 * sigma_thomson * c * U_B /nu1
    return const

def calc_jcyclo(nu, gamma):
    '''
    This function calculates the cyclotron emissivity as a function of emitted frequency and lorentz factor of electrons.
    It implements equation 16 of Lucchini et al.

    Parameters:
    -------------
    nu : float
         Emitted frequency in co-moving frame (in Hz)
    gamma: float
           Electron lorentz factor

    Returns:
    ---------
    Cyclotron emissivity for a specific frequency and lorentz factor (float)
    '''
    Q = get_Q(gamma)
    Q_term = Q**2/(1 + 3*Q*Q)
    jconst = const_cycloemissivity()
    nu1 = calc_Larmorfreq()
    expterm = np.exp(2*(1-nu/nu1)/(1 + 3*Q**2))
    return jconst * Q_term * expterm

def cyclotronlimits():
    '''
    Calculates the range of frequencies in which cyclotron is emitted, which essentially corresponds to gamma^2 times the characteristic gyrofrequency.
    Returns the minimum and maximum frequency of radiation.
    '''
    numin = gamma_min**2 * calc_Larmorfreq()
    numax = 4 * calc_Larmorfreq() #corresponds to gamma=2, upper limit of cyclotron radiation
    return numin, numax

def synchrotronlimits():
    '''
    Calculates the range of frequencies in which cyclotron is emitted, which essentially corresponds to gamma^2 times the characteristic gyrofrequency given in lectures.
    Returns the minimum and maximum frequency of radiation.
    '''
    numin = 3/2 * 2**2 * e * B / (m_e * c)
    numax = 3/2 * gamma_max**2 * e * B / (m_e * c)  #our gamma_max is 100
    return numin, numax

def calc_totalcycloemissivity(nu, gamma_array):
    '''
    This function integrates cyclotron emissivity over an array of lorentz factors, as a result of which we get the total emission at a particular frequency.
    This has to still be corrected for extinction in the medium.

    Parameters:
    ------------
    nu : float
         Emitted frequency in co-moving frame (in Hz)
    gamma_array : numpy.ndarray of floats
                  array of lorentz factors (for us, ranging from 1.1 to 2, linearly spaced)

    Returns:
    ----------
    integral : float
               cyclotron emissivity as a function of frequency
    '''
    dqdgamma = get_dQdgamma(gamma_array)
    N_Q = get_NQ(gamma_array)
    N_gamma = get_dQdgamma(gamma_array) * get_NQ(gamma_array)
    jcyclo = calc_jcyclo(nu, gamma_array)
    integral = np.trapz(N_gamma*jcyclo)
    return integral

def calc_absgamma(p):
    '''This function calculates the product of the terms in the absorption coeffient involving the gamma function. It uses the scipy package's gamma
    function to do so. The function serves as an intermediary to enable easier computation of the absorption coefficient of the blob.'''
    return scipy.special.gamma((3*p + 2)/12)*scipy.special.gamma((3*p+22)/12)

def calc_C(p, gamma_array, eta, Mdot):
    '''
    Calculates the quantity C associated with the electron power law distribution and is used in optical depth and power calculation.

    Parameters:
    ------------
    1. p: float
          electron power law index
    2. gamma_array: numpy.ndarray
                    array of lorentz factors that electrons can have
    3. eta: float
            efficiency of accretion
    4. Mdot: float
             Mass accretion rate

    Returns:
    ---------
    float: Quantity C calculated using the total electron energy density and that obtained by integrating the power law energy distribution.
    '''
    U_e = calc_Utot(eta, Mdot) / 2
    E_array = gamma_array * m_e * c**2
    integral = np.trapz(E_array**(-p + 1))
    return U_e / integral


def calc_Chat(p, gamma_array, eta, Mdot):
    '''
    This is the C used in the absorption coefficient equation, called C_hat. It is the above C/ (m_e * c**2)
    '''
    C = calc_C(p, gamma_array, eta, Mdot)
    return C / (m_e * c**2)

def calc_ne(p, gamma_array, eta, Mdot):
    '''
    This function computes the electron number density in the corona assuming equipartition holds.

    Parameters:
    ------------
    1. p: float
          electron power law index
    2. gamma_array: numpy.ndarray
                    array of lorentz factors that electrons can have
    3. eta: float
            efficiency of accretion
    4. Mdot: float
             Mass accretion rate

    Returns:
    ----------
    float: Electron number density in the corona (in cm^-3) using n_e = C * integral of the power law electron distribution
    '''
    C = calc_C(p, gamma_array, eta, Mdot)
    E_array = gamma_array * m_e * c**2
    integral = np.trapz (E_array ** (-p))
    return C * integral

def calc_alpha(p, gamma_array, eta, Mdot, nu):
    '''
    Calculates the absorption coefficient for a particular frequency using the formula given in Rybicki and Lightman (eq 6.53).

    Parameters:
    ------------
    1. p: float
          electron power law index
    2. gamma_array: numpy.ndarray
                    array of lorentz factors that electrons can have
    3. eta: float
            efficiency of accretion
    4. Mdot: float
             Mass accretion rate
    5. nu: float
           frequency at which absorption coefficient is to be calculated

    Returns:
    ----------
    float: Absorption coefficient calculated using the provided formula.
    '''
    C_hat = calc_Chat (p, gamma_array, eta, Mdot)
    const = np.sqrt(3) * e**3 / (8 * np.pi * m_e)
    pterm = 3 * e / (2 * np.pi * m_e**3 * c**5)
    pterm = pterm **(p/2)
    Bterm = B**((p+2)/2)
    gammaterm = calc_absgamma(p)
    nuterm = nu**(-(p+4)/2)
    alpha = C_hat * const * pterm * Bterm * gammaterm * nuterm
    return alpha

def calc_opticaldepth(p, gamma_array, eta, Mdot, nu):
    '''
    Calculates the optical depth of the corona using the formula: Optical depth = alpha * r * 2 / 3.
    This comes from integrating the absorption coefficient across the diameter of the corona.

    Parameters:
    -----------
    1. p: float
          electron power law index
    2. gamma_array: numpy.ndarray
                    array of lorentz factors that electrons can have
    3. eta: float
            efficiency of accretion
    4. Mdot: float
             Mass accretion rate
    5. nu: float
           frequency at which absorption coefficient is to be calculated

    It utilizes the function calc_alpha defined previously.

    Returns:
    ---------
    float: Optical depth calculated using the provided formula.
    '''
    alphanu = calc_alpha(p, gamma_array, eta, Mdot, nu)
    R = 10 * r_g
    return alphanu* R * 2/3

def synchemissivity(p, gamma_array, eta, Mdot, nu):
    '''
    This function calculates the emissivity of synchrotron self absorption at a particular frequency using the formula provided in Rybicki and Lightman (eq 6.36).

    Parameters:
    -----------
    1. p: float
          electron power law index
    2. gamma_array: numpy.ndarray
                    array of lorentz factors that electrons can have
    3. eta: float
            efficiency of accretion
    4. Mdot: float
             Mass accretion rate
    5. nu: float
           frequency at which absorption coefficient is to be calculated

    Returns:
    ---------
    jnu: float
         Emissivity calculated using the provided formula.
    '''
    C = calc_C (p, gamma_array, eta, Mdot)
    Bterm = B**(1+(p-1)/2)
    gammaterm = scipy.special.gamma(p/4 + 19/12) * scipy.special.gamma(p/4 - 1/12)
    const = np.sqrt (3) * e**3 / (2 * np.pi * m_e * c**2 * (p+1))
    const2 = (3 * e / m_e * c) ** ((p-1)/2)
    nuterm = (2 * np.pi * nu)**(-(p-1)/2)
    jnu = C * B * gammaterm * const * const2 * nuterm
    return jnu

def calc_Inu(p, gamma_array,eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, nu_array):
    '''
    This function uses functions defined previously to get the total specific intensity from cyclotron and synchrotron self absorption in the corona.

    Parameters:
    -----------
    1. p: float
          electron power law index
    2. gamma_array: numpy.ndarray
                    array of lorentz factors that electrons can have
    3. eta: float
            efficiency of accretion
    4. Mdot: float
             Mass accretion rate
    5. cyclo_gamma_array: numpy.ndarray of floats
                          array of lorentz factors for cyclotron

    5. synch_gamma_array: numpy.ndarray of floats
                          array of lorentz factors for synchrotron

    5. cyclonu_array: numpy.ndarray of floats
                      frequency array for cyclotron, obtained from cyclotronlimits function

    5. nu_array: numpy.ndarray of floats
                 frequency array for synchrotron, obtained from synchrotronlimits function

    Returns:
    ---------
    list of floats: Specific intensity for cyclosynchrotron as a function of frequency calculated using previous functions.
    '''
    Inu = []
    for nu in cyclonu_array:
        cycloem = calc_totalcycloemissivity(nu, cyclo_gamma_array)
        alphanu = calc_alpha(3.3, cyclo_gamma_array, eta, Mdot, nu)  #slightly lower power law assumed for low gamma
        tau = calc_opticaldepth(3.3, cyclo_gamma_array, eta, Mdot, nu)
        Inu.append(cycloem/alphanu * (1-np.exp(-tau)))  #Inu = Source function * (1- e^-(opticaldepth))
    for nu in nu_array:
        synchem = synchemissivity(p, gamma_array, eta, Mdot, nu)
        alphanu = calc_alpha(p, gamma_array, eta, Mdot, nu)
        tau = calc_opticaldepth(p, gamma_array, eta, Mdot, nu)
        Inu.append(synchem/alphanu * (1-np.exp(-tau)))
    return Inu

def calc_Fnu_Surface(p, gamma_array, eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, nu_array):
    '''
    This function obtains the flux at the surface of the corona by integrating the specific intensity over the solid angle.

    Returns
    --------
    list of floats: Flux at the surface of the corona from cyclosynchrotron self absorption
    '''
    Inu_surface = calc_Inu(p, gamma_array,eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, nu_array)
    Fnu_surface = np.array(Inu_surface) * 2 * np.pi
    return Fnu_surface

def calc_Fnu_Earth(p, gamma_array, eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, nu_array):
    '''
    Here, the flux at the surface of the corona is translated to that at Earth by accounting for geometric dilution.
    '''
    Fnu_surface = calc_Fnu_Surface(p, gamma_array, eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, nu_array)
    R = 10 * r_g
    Fnu_Earth = Fnu_surface * R**2 / D**2 #D is the distance to V404 Cyg
    return Fnu_Earth

def N_nu (p, gamma_array,eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, nu_array):
    """
    Calculates the number of photons per frequency bin for the Cyclotron-Synchrotron spectrum

    Returns
    -------------
    N_nu: numpy array
    - Number of photons corresponding to the frequency bins
    """
    Inu = calc_Inu (p, gamma_array,eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, nu_array)
    combinednu_array = np.concatenate([cyclonu_array, nu_array])
    Int_array = np.array (Inu)
    N_nu = np.zeros(len(Int_array))
    for i in range(0,len(combinednu_array)):
        Eph = h * combinednu_array[i]
        N_nu[i] = Int_array[i]/Eph
    return N_nu

def sample_photons(No_samples,N_nu,nu_array):
    """
    Samples a photon from the synchrotron spectrum, calculates its corresponding energy and returns the neergy value

    Parameters
    -------------
    No_samples: int
    - number of samples needed
    N_nu: numpy array
    - Photon number distribution
    nu_array: numpy array
    - Frequencies corresponding to the number distribution of photons

    Returns
    -------------

    """
    N_nu_pdf = N_nu / np.sum(N_nu)
    cdf = np.cumsum(N_nu_pdf)
    random_values = np.random.uniform(0, 1, size=No_samples)
    sampled_synch = np.interp(random_values, cdf, nu_array) * h
    return sampled_synch

def sample_photons_wrapper(mc_parms, number=1):
    """
    Wrapper function to implement the photon sampling function into the Monte Carlo code
    Parameters
    -------------
    mc_parms: dict
    - Dictionary with pre-defined values
    number: int
    - number of samples to be taken
    Returns
    -------------
    sample_photons: numpy array
    - Energies corresponding to the sampled photons
    """
    N_photons = N_nu(3.5, gamma_array,eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, synchnu_array)
    return sample_photons(number,N_photons,combined_nuarray)


<h2>Calling Functions to get Flux From Cyclo-synchrotron (Raghavan)

In [7]:
#Creating the lorentz factor and frequency arrays for cyclosynchrotron self absorption
synch_gamma_array = np.logspace(np.log10(gamma_min), np.log10(gamma_max), 100)
cyclo_gamma_array = np.linspace(1.1,2,100)
gamma_array = np.concatenate([cyclo_gamma_array, synch_gamma_array])
em_list = []
c_min, c_max = cyclotronlimits()
cyclonu_array = np.logspace(np.log10(c_min), np.log10(c_max), 100)
s_min, s_max = synchrotronlimits()
synchnu_array = np.logspace(np.log10(s_min), np.log10(s_max),100)
eta, Mdot = calc_etaMdot()

Intensity_surface = calc_Inu(3.5, gamma_array, eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, synchnu_array)
combined_nuarray = np.concatenate([cyclonu_array,synchnu_array])
Flux_E = calc_Fnu_Earth(3.5, gamma_array, eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, synchnu_array)

#Next we calculate the total number of photons from cyclosynchrotron self absorption, used to normalize the flux from SSC
Total_photons = np.array(Intensity_surface)* 4 * np.pi * R**2/ h

N_photons = N_nu(3.5, gamma_array,eta, Mdot, cyclo_gamma_array, synch_gamma_array, cyclonu_array, synchnu_array)

<h2>Inverse Compton Scattering of Black Body and Cyclsynchrotron (From Sebastian Heinz, modified by Shashanth and Raghavan)

In [ ]:
mc_parms={'n_photons':1000,            # Number of photons to sample
          'kt_seeds':k_B * 1e8,            # Photon energy used for normalization - does not matter for us
          'H':1e7,                       # Height of Corona
          'tau':1,                       # Optical depth
          'kt_electron':k_B * T_corona,          # Energy of electron distribution - does not matter for us
          'v_dist':wrapper_sample_maxwell_juttner,      # name of electron distribution function
          'hnu_dist':wrapper_sample_planck,    # name of photon distribution function
          'gamma_min':1.1,
          'gamma_max':100,
          'T_corona': 1e8,
          'T_BB':1e7,
         }
hnu_scattered_BB,hnu_seeds_BB=plot_mc(mc_parms)

mc_parms2={'n_photons':1000,            # Number of photons to sample
          'kt_seeds':k_B * 1e8,            # Photon energy used for normalization - does not matter for us
          'H':1e7,                       # say H ~ R, and R ~ 100 R_g ~ 3e7 cm
          'tau': 2,                       # Optical depth
          'kt_electron':k_B * T_corona,          # Energy of electron distribution - does not matter for us
          'v_dist':wrapper_sample_maxwell_juttner,      # name of photon distribution function
          'hnu_dist':sample_photons_wrapper,    # name of photon distribution function
          'gamma_min':1.1,
          'gamma_max':100,
          'T_corona': 1e8,
          'T_BB':1e7,
         }
hnu_scattered_SSC,hnu_seeds_SSC=plot_mc(mc_parms2)

Fraction of escaping photons: 4.870e-01

Compton y parameter: 1.246e+00



<h1>Function for scaling and plotting MC Output as a flux spectrum (Shashanth)



In [ ]:
def plothist(hnu_scattered_true, factor=1):
    """
    Calculates the flux output from a given array of exiting energies of photons.

    Parameters
    ---------------------
    hnu_scattered_true: numpy array
    - The true energy values in ergs of the scattered photons escaping the corona

    Returns
    ---------------------
    freq_centres: numpy array
    - array corresponding to frequencies of the escaping phtons
    flux_scattered: numpy array
    -
    """
    # Define logarithmic bins based on min and maximum value of hnu_scattered
    mybins = np.logspace(np.log10(min(hnu_scattered_true)), np.log10(max(hnu_scattered_true)), num=201)

    # Convert into histogram
    counts, mybins = np.histogram(hnu_scattered_true, bins=mybins)
    bin_centres = (mybins[1:] + mybins[:-1]) / 2 # Bin centres needed to plot
    bin_energies = counts * bin_centres # Converting photon numbers into energies
    freq_centres = bin_centres / h # Converting energy values into frequency
    flux_scattered = bin_energies * factor * 1e26 / (4 * np.pi * D**2)
    return freq_centres, flux_scattered




<h2>Plotting the Sampled Energy Distribution of Electrons

In [ ]:
E_dist_maxwell_juttner()

# Plotting input photon and electron fields (Shashanth and Raghavan)

In [ ]:
nus_p, f_p = calc_planck(T_disk)
plt.plot(combined_nuarray, np.array(Intensity_surface), label='Cyclo-synchrotron')
plt.plot(nus_p, f_p, label='Disk')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Intensity (erg/s/cm^2/Hz/sr)')
plt.ylim(1e-20, 1e7)
plt.xlim(1e5, 1e20)
plt.title('Input Photon Fields')
plt.legend()
plt.show()

In [ ]:
gammas = np.linspace(gamma_min, gamma_max, 1000)
juttnerdist = maxwell_juttner(gammas) * m_e * c**2
plt.plot(gammas, juttnerdist)
plt.xlabel('Lorentz Factor Gamma')
plt.ylabel('Electron energy')
plt.title('Input Electron Field')
plt.xscale('log')
plt.yscale('log')
plt.show()

<h3>Calling the functions to plot the Flux spectra along with data (Raghavan)

In [ ]:
freq_centres_BB, flux_scattered_BB = plothist(hnu_scattered_BB, N_photons_actual/mc_parms['n_photons'])
freq_centres_SSC, flux_scattered_SSC = plothist(hnu_scattered_SSC, Total_photons / mc_parms2['n_photons'])
plt.plot(freq_centres_BB, flux_scattered_BB, '.', label='BB IC')
plt.plot(freq_centres_SSC, flux_scattered_SSC, '.', label='SSC')

# True data extracted from literature
RadioOP_X = [1418244058,2427806153,1537341773,5153025087,8359474712,5016364723,5016364723,8359474712,8587211115,1.28514E+13,3.97394E+13,6.44672E+13,1.36832E+14,1.83904E+14,2.34234E+14,4.58647E+14,4.58647E+14,5.38912E+14,7.05097E+14,8.74247E+14,1.14384E+15]
RadioOP_Y = [0.335343985,0.326749365,0.180979316,0.1991956,0.199811699,0.380238682,0.495393054,0.527105452,0.370493445,0.58582852,2.491333247,4.368438097,10.60208239,17.00067238,15.13815172,11.66922392,9.223894226,7.087086834,3.404582893,0.410687091,0.024475086]
XrayData_X = [1.62445E+17,1.82992E+17,1.9926E+17,2.05304E+17,2.17969E+17,2.19611E+17,2.36205E+17,2.44554E+17,2.55146E+17,2.74433E+17,2.91354E+17,3.05319E+17,3.21501E+17,3.50366E+17,3.78321E+17,4.30224E+17,4.53012E+17,4.85123E+17,5.28711E+17,5.65994E+17,6.16926E+17,7.07772E+17,7.71053E+17,8.40412E+17,8.92117E+17,9.88664E+17,1.10498E+18]
XrayData_Y = [59666.47395,112581.6234,168131.3045,185874.9608,216086.0187,387501.5695,428529.7377,350859.9299,473916.6678,515422.2987,609277.6685,769860.2125,732546.62,708945.8991,957832.2578,1132811.953,1096042.754,1253181.667,1173003.481,1611368.635,1410896.184,1484795.043,1726447.603,1537091.899,1942392.561,2335449.784,3002038.205]

# Plotting actual data
plt.scatter(RadioOP_X,RadioOP_Y, label='Data for Radio to X-Ray',color='red')
plt.scatter(XrayData_X, XrayData_Y, label='Data for X-Ray', color='blue', marker='o')
plt.plot(combined_nuarray, np.array(Flux_E)/1e-26, label='Cyclo-Synchrotron')
plt.xlim(1e7,1e20)

plt.title('Flux Density Profiles')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Flux Observed at Earth (mJy)')
plt.legend()
plt.show()

Adding up contributions of individual processes (Raghavan, but took some code from Stack Overflow)

In [ ]:
def mask_between_a_b(array, a, b):
    '''
    Generate a boolean mask for elements in the array that fall between a and b.

    This function generates a boolean mask for elements in the input array that lie
    within the range defined by 'a' and 'b' (inclusive).

    Parameters:
    array : array_like
        Input array to be masked.
    a : float
        Lower bound of the range.
    b : float
        Upper bound of the range.

    Returns:
    numpy.ndarray-Boolean mask where True indicates elements in the array that fall between 'a' and 'b' (inclusive)
    '''
    #This has been taken from stack overflow
    return np.logical_and(a <= array, array <= b)

In [ ]:
x1 = combined_nuarray
y1 = np.array(Flux_E)/1e-26
x2 = freq_centres_SSC
y2 = flux_scattered_SSC

xvals1 = np.linspace(x1[0], x1[-1], 1000)
yinterp1 = np.interp(xvals1, x1, y1)

xvals2 = np.linspace(x2[0], x2[-1], 1000)
yinterp2 = np.interp(xvals2, x2, y2)

a = x1[0]
b = x2[0]
c = x1[-1]
d = x2[-1]
x1_mask = mask_between_a_b(x1,a,b)
x2_mask = mask_between_a_b(x2,c,d)

x1m = x1[x1_mask]
y1m = y1[x1_mask]
x2m = x2[x2_mask]
y2m = y2[x2_mask]

xvals = np.linspace(a, d, 1000)
x1_and_x2_intersection_mask = (np.logical_and(b <= xvals, xvals <= c))
xinter = xvals[x1_and_x2_intersection_mask]

yi1 = yinterp1[x1_and_x2_intersection_mask]
yi2 = yinterp2[x1_and_x2_intersection_mask]

x = list(x1m) + list(xinter) + list(x2m)
y = list(y1m) + list(yi1+yi2) + list(y2m)
plt.plot(x, y, '.',label='Cyclo-Synchrotron + SSC')
plt.plot(freq_centres_BB, flux_scattered_BB,'.',label='BB IC')
plt.scatter(RadioOP_X,RadioOP_Y, label='Data for Radio to X-Ray',color='red')
plt.scatter(XrayData_X, XrayData_Y, label='Data for X-Ray', color='blue', marker='o')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()
